# Fitting a GMM to the AdVitam Dataset


<hr style="border-top: 1px solid white;">


## Preamble


Python Libraries


In [1]:
import numpy as np
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV
)

<br></br>


Custom Functions


In [3]:
# in order of appearance
from useful_functions.check_for_missing_data import check_for_missing_data

# driving data
from useful_functions.driving_data.dd_dictionary import create_dd_dictionary
from useful_functions.driving_data.process_driving_data import processing_driving_data
from useful_functions.takeover_dataframe import create_takeover_timestamps

# physio data
from useful_functions.physio_data.pd_dictionary import create_pd_dictionary
from useful_functions.physio_data.process_physio_timestamps import process_physio_timestamps
from useful_functions.physio_data.preprocess_physio_data import preprocess_physio_data

from useful_functions.demographic_data.process_driver_demographic_data import (
    process_driver_demographic_data,
)

from useful_functions.construct_observations import construct_observations

<br></br>

Storing the folder paths to raw data


In [4]:
driving_data_folder = "../AdVitam/Exp2/Raw/Driving"
physio_data_folder = "../AdVitam/Exp2/Raw/Physio/Txt"

<br></br>

Storing a list of driver files to exclude

In [5]:
drivers_to_exclude = check_for_missing_data(driving_data_folder, physio_data_folder)
drivers_to_exclude.extend(["NST77", "NST11", "ST22", "NST87", "ST14", "ST12", "NST73", "ST10"])

`check_for_missing_data()` returns a list of any driver that is _not_ in both the `driving` and `physio` folders.

Participants: ST22, NST87, ST14, ST12, NST73 and ST10 seem to have issues with there physiological data

<br></br>
<br></br>

# Importing Data + Preprocessing


---


<br>


## Driving Data


<hr style="border-top: 1px dashed white; border-bottom: 0px">


### Data Description


**Driver Data:**
| Feature | Description | Notes |
| --- | --- | --- |
| Time | Time elapsed since the software was launched (in seconds) | |
| EngineSpeed | Engine speed (in rpm) | Removed |
| GearPosActual | Current gear | Removed |
| GearPosTarget | Next planned gear | Removed |
| AcceleratorPedalPos | Position of gas pedal. | Recording problem, Removed |
| DeceleratorPedalPos | Position of brake pedal. | Recording problem, Removed |
| SteeringWheelAngle | Steering wheel angle (in degrees) | |
| VehicleSpeed | Vehicle speed (in mph) | |
| Position X | Vehicle position along the x-axis in the simulated driving environment | |
| Position Y | Vehicle position along the y-axis in the simulated driving environment | |
| Position Z | Vehicle position along the z-axis in the simulated driving environment | |
| Autonomous Mode (T/F) | Autonomous pilot status. The driver is in control of the car when the value of the column "Autonomous Mode (T/F)" is False | True = Activated, False = Deactivated (driver in control) |
| Obstacles | Events that occurred during the driving simulation. | See Below |


<br></br>


**Obstacles:**

| Event         | Description                                                                                                                                                                                             |
| ------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| TriggeredObsX | Time at which each takeover request was triggered by the experimenter                                                                                                                                   |
| Obs1          | Deer                                                                                                                                                                                                    |
| Obs2          | Traffic cone                                                                                                                                                                                            |
| Obs3          | Frog                                                                                                                                                                                                    |
| Obs4          | Traffic cone                                                                                                                                                                                            |
| Obs5          | False alarm (x2)                                                                                                                                                                                        |
| Detected      | Time at which the driver pressed the steering wheel button to notify he/she understood the situation. |


<br></br>


### Driving Data Dictionary


Creates a dictionary of the raw driving data files.


In [6]:
driving_data_dictionary = create_dd_dictionary(driving_data_folder, drivers_to_exclude)

`create_dd_dictionary()` stores every file in the `driving_data_folder` in a dictionary

In [8]:
# example usage
driving_data_dictionary['NST01'].head()

,Time,SteeringWheelAngle,VehicleSpeed,Autonomous Mode (T/F),Obstacles
0,102.0442,-51.0,0.0000,False,NaN
1,102.0442,-51.0,0.0000,False,NaN
2,102.0642,-51.0,11.1829,False,NaN
3,102.0739,-51.0,11.1829,False,NaN
4,102.0782,-51.0,11.1829,False,NaN


<br></br>


Processing driving data


In [9]:
# Fitting a Label Encoder to the `Obstacles` column
driver_data = driving_data_dictionary["NST01"]
driver_data = driver_data.fillna("Nothing")
enc = preprocessing.LabelEncoder()
enc.fit(driver_data["Obstacles"])

# Processing the driving data
driving_data_dictionary = processing_driving_data(driving_data_dictionary, enc)

In [10]:
# example usage
driving_data_dictionary["NST01"].head()

,Time,SteeringWheelAngle,VehicleSpeed,Autonomous Mode (T/F),Obstacles
0,0 days 00:01:42.044200,-51.0,0.0000,False,1
1,0 days 00:01:42.054200,-51.0,0.0000,False,1
2,0 days 00:01:42.064200,-51.0,11.1829,False,1
3,0 days 00:01:42.074200,-51.0,11.1829,False,1
4,0 days 00:01:42.084200,-51.0,7.0143,False,1


A label Encoder is fit to the `Obstacles` of driver `NST01`

`processing_driving_data()`label encodes the `Obstacles` for every driver and resamples the driving data to 10ms (or 100Hz)

<br></br>


Creating driving data takeover timestamps


In [11]:
driving_timestamps = create_takeover_timestamps(driving_data_dictionary, enc)

`create_takeover_timestamps()` extracts timestamps from each driver takeover.

Driving Timestamps include:
- `TriggeredObsX` : When the takeover request for obstacle X was triggered
- `TakeoverObsX`: When the driver tookover the vehicle
- `ReleaseObsX`: When the driver released control of the vehicle to the automation
- `TOTObsX`: The amount of time it took for the driver to takeover for obstacle X (`TakeoverObsX` - `TriggeredObsX`)


In [12]:
driving_timestamps.head()

,subject_id,TriggeredObs1,TakeoverObs1,ReleaseObs1,TOTObs1,TriggeredObs2,TakeoverObs2,ReleaseObs2,TOTObs2,TriggeredObs3,...,ReleaseObs3,TOTObs3,TriggeredObs4,TakeoverObs4,ReleaseObs4,TOTObs4,TriggeredObs5,TakeoverObs5,ReleaseObs5,TOTObs5
0,NST01,0 days 00:05:11.974200,0 days 00:05:18.804200,0 days 00:05:28.764200,0 days 00:00:06.830000,0 days 00:09:11.494200,0 days 00:09:13.964200,0 days 00:09:23.654200,0 days 00:00:02.470000,0 days 00:10:50.094200,...,0 days 00:10:54.554200,0 days 00:00:04.080000,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,ST02,0 days 00:08:03.979300,0 days 00:08:08.999300,0 days 00:08:17.339300,0 days 00:00:05.020000,0 days 00:06:03.149300,0 days 00:06:06.569300,0 days 00:06:09.769300,0 days 00:00:03.420000,0 days 00:14:38.599300,...,0 days 00:14:44.779300,0 days 00:00:04.560000,0 days 00:17:24.939300,0 days 00:17:29.289300,0 days 00:17:33.199300,0 days 00:00:04.350000,NaT,NaT,NaT,NaT
2,NST03,0 days 00:16:04.013200,0 days 00:16:08.633200,0 days 00:16:41.013200,0 days 00:00:04.620000,0 days 00:12:48.623200,0 days 00:12:51.843200,0 days 00:13:24.443200,0 days 00:00:03.220000,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,ST04,0 days 00:19:23.934300,0 days 00:19:36.624300,0 days 00:19:54.174300,0 days 00:00:12.690000,0 days 00:13:29.504300,0 days 00:13:32.174300,0 days 00:13:39.614300,0 days 00:00:02.670000,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,NST05,0 days 00:10:02.164780,0 days 00:10:04.474780,0 days 00:10:06.294780,0 days 00:00:02.310000,0 days 00:15:16.364780,0 days 00:15:31.474780,0 days 00:15:36.064780,0 days 00:00:15.110000,0 days 00:17:52.614780,...,0 days 00:18:00.284780,0 days 00:00:03.340000,0 days 00:07:21.274780,0 days 00:07:24.604780,0 days 00:07:27.654780,0 days 00:00:03.330000,NaT,NaT,NaT,NaT


<br></br>
<br></br>


## Physiological Signals & Markers


<hr style="border-top: 1px dashed white; border-bottom: 0px">


### Data Description


**Signals:**

| Feature | Description            | Notes  |
| ------- | ---------------------- | ------ |
| min     | Time Elapsed           |        |
| ECG     | Electrocardiogram      | 1000Hz |
| EDA     | Electrodermal Activity | 1000Hz |
| RESP    | Resperatory            | 1000Hz |


<br></br>


**Markers:**

Contains the timestamps for each period of the experiment.

- Training1 = Baseline phase
- Training2 = Practice phase in the driving simulator
- Driving = Main driving session in conditionally automated driving.

Be careful, the timestamps are here in seconds while they are in minutes in the raw data.


<br></br>


**Timestamps:**

Time elapsed (in seconds) between the start of the main driving session and the appearance of the obstacles.

- TrigObsX: the time when the driver pressed the button to report having understood the situation
- DetObsX: and the time when the driver actually took over control
- RepObsX: X corresponds to one of obstacle or the false alarm.


<br></br>


### Physio data dictionary


Creates a dictionary of the raw physiological data and their markers


In [13]:
phsyiological_data_dictionary = create_pd_dictionary(physio_data_folder, drivers_to_exclude)

`create_pd_dictionary()` stores every file in the `physio_data_folder` in a dictionary

In [16]:
phsyiological_data_dictionary["NST01"].head()

,min,CH1,CH2,CH3
0,0.000000,15.7639,7.09503,-0.310669
1,0.000017,15.7639,7.08344,-0.310974
2,0.000033,15.7639,7.07001,-0.311584
3,0.000050,15.7623,7.05414,-0.311890
4,0.000067,15.7593,7.03644,-0.312500


In [17]:
phsyiological_data_dictionary["NST01-markers"].head()

,Marker Index:,Time(sec.):,Label:
0,Event 1:,4.095,"Training 1 Start, 08:13:59"
1,Event 2:,305.320,"Training 1 End, 08:19:00"
2,Event 3:,534.260,"Training 2 Start, 08:22:49"
3,Event 4:,794.405,"Training 2 End, 08:27:09"
4,Event 5:,973.685,"Driving Start, 08:30:08"


<br></br>


Processing the Physiological data


In [18]:
phsyiological_data_dictionary = preprocess_physio_data(phsyiological_data_dictionary)

`process_physio_data()` resamples the data to 10ms (100Hz) and then segments the data into each experimental phase (Baseline, Training, Driving)


<br></br>


### Physio timestamps


A dataframe to store the trigger time, takeover time, release time, and TOT for each obstacle, for every driver. Similar to `driver_timestamps`.


In [19]:
physio_timestamps = pd.read_csv(
    "../AdVitam/Exp2/Preprocessed/Physio and Driving/timestamps_obstacles.csv"
)

In [20]:
physio_timestamps.head()

,subject_id,label_st,TrigObsDeer,DetObsDeer,RepObsDeer,TrigObsCone,DetObsCone,RepObsCone,TrigObsFrog,DetObsFrog,RepObsFrog,TrigObsCan,DetObsCan,RepObsCan,TrigObsFA1,DetObsFA1,RepObsFA1,TrigObsFA2,DetObsFA2,RepObsFA2
0,NST1,0,176.7051,179.0932,183.5238,416.2140,418.5109,418.6902,514.8157,518.8456,NaN,786.6408,792.8591,NaN,983.6240,NaN,NaN,1082.2450,1086.3630,NaN
1,ST2,1,230.7565,234.5881,235.7780,109.9334,112.2556,113.3516,625.3827,628.3633,629.9416,791.7203,793.7173,796.0673,357.7144,360.2164,NaN,468.5158,470.4113,NaN
2,NST3,0,815.2040,818.4693,819.8244,619.8088,621.8167,623.0322,259.4712,263.7890,NaN,1027.4009,1030.7859,NaN,378.3409,NaN,NaN,1115.1749,1122.8189,NaN
3,ST4,1,1040.3619,1042.3819,1053.0479,685.9281,NaN,NaN,287.3724,289.5112,NaN,119.9266,120.5600,NaN,410.9882,462.7615,NaN,886.9369,889.8139,NaN
4,NST5,0,428.9613,430.7424,431.2726,743.1664,744.4617,758.2736,899.4186,900.7246,902.7536,268.0716,269.9875,271.4027,143.6314,144.7424,NaN,629.1736,630.4212,NaN


<br></br>


### Processing the Physio Timestamps

Steps:

1. Change column names to match driving timestamps
1. Remove preselected drivers
1. Reformat subject id to match
1. Transfrom timestamps into timedelta objects


In [21]:
physio_timestamps = process_physio_timestamps(physio_timestamps, drivers_to_exclude)

In [22]:
physio_timestamps.head()

,subject_id,TriggeredObs1,TakeoverObs1,TriggeredObs2,TakeoverObs2,TriggeredObs3,TakeoverObs3,TriggeredObs4,TakeoverObs4,TriggeredObs5,TakeoverObs5,TriggeredObs6,TakeoverObs6,TOTObs1,TOTObs2,TOTObs3,TOTObs4,TOTObs5,TOTObs6
0,NST01,0 days 00:02:56.705100,0 days 00:03:03.523800,0 days 00:06:56.214000,0 days 00:06:58.690200,0 days 00:08:34.815700,NaT,0 days 00:13:06.640800,NaT,0 days 00:16:23.624000,NaT,0 days 00:18:02.245000,NaT,0 days 00:00:06.818700,0 days 00:00:02.476200,NaT,NaT,NaT,NaT
1,ST02,0 days 00:03:50.756500,0 days 00:03:55.778000,0 days 00:01:49.933400,0 days 00:01:53.351600,0 days 00:10:25.382700,0 days 00:10:29.941600,0 days 00:13:11.720300,0 days 00:13:16.067300,0 days 00:05:57.714400,NaT,0 days 00:07:48.515800,NaT,0 days 00:00:05.021500,0 days 00:00:03.418200,0 days 00:00:04.558900,0 days 00:00:04.347000,NaT,NaT
2,NST03,0 days 00:13:35.204000,0 days 00:13:39.824400,0 days 00:10:19.808800,0 days 00:10:23.032200,0 days 00:04:19.471200,NaT,0 days 00:17:07.400900,NaT,0 days 00:06:18.340900,NaT,0 days 00:18:35.174900,NaT,0 days 00:00:04.620400,0 days 00:00:03.223400,NaT,NaT,NaT,NaT
3,ST04,0 days 00:17:20.361900,0 days 00:17:33.047900,0 days 00:11:25.928100,NaT,0 days 00:04:47.372400,NaT,0 days 00:01:59.926600,NaT,0 days 00:06:50.988200,NaT,0 days 00:14:46.936900,NaT,0 days 00:00:12.686000,NaT,NaT,NaT,NaT,NaT
4,NST05,0 days 00:07:08.961300,0 days 00:07:11.272600,0 days 00:12:23.166400,0 days 00:12:38.273600,0 days 00:14:59.418600,0 days 00:15:02.753600,0 days 00:04:28.071600,0 days 00:04:31.402700,0 days 00:02:23.631400,NaT,0 days 00:10:29.173600,NaT,0 days 00:00:02.311300,0 days 00:00:15.107200,0 days 00:00:03.335000,0 days 00:00:03.331100,NaT,NaT


<br></br>


## Driver Demographic Data


<hr style="border-top: 1px dashed white; border-bottom: 0px">


### Data Description


**Driver Demographic Data Description:**
| Feature | Description | Note |
| --- | --- | --- |
| code | Code of driver Secondary Task (ST) vs No ST (NST) + unique id (1,2,...) | In the form (ST/NST)# |
| date | Day of data collection | Removed |
| time | Hour of data collection | Removed |
| condition | Experimental condition for mental workload | Removed (contained in driver code |
| sex | driver sex | |
| age | Age of drivers in years | |
| mothertongue | drivers first language | |
| education | Highest education degree | |
| driving_license | Year of obtenstion of driving license | |
| km_year | Number of kilometers covered per year in average | |
| accidents | Number of accidents during the last 3 years | |
| nasa_tlx_N | Answer to the NASA TLX for question N | Removed |
| danger_O | Subjective ranking of the danger of obstacle O | Removed |
| realism_O | Subjective ranking of the realism of obstacle O | Removed |
| sart_N_O | Subjective answer to the sart for question N related to obstacle O | Removed |
| demand_O | Demands on attentional resources (complexity, variability, and instability of the situation) | Removed |
| supply_O | Supply of attentional resources (division of attention, arousal, concentration, and spare mental capacity) | Removed |
| understanding_O | Understanding of the situation (information quantity, information quality and familiarity). | Removed |


<br></br>


### Driver Demographic Data


Grabbing the driver demographic data


In [23]:
driver_demographic_data = pd.read_csv(
    "../AdVitam/Exp2/Preprocessed/Questionnaires/Exp2_Database.csv",
    usecols=[
        "code",
        "sex",
        "age",
        "mothertongue",
        "education",
        "driving_license",
        "km_year",
        "accidents",
    ],
)

In [24]:
driver_demographic_data.head()

,code,sex,age,mothertongue,education,driving_license,km_year,accidents
0,NST1,1,19,1,1,2017,200,1
1,ST2,1,19,1,1,2017,5000,0
2,NST3,1,19,1,1,2017,1000,0
3,ST4,1,21,3,2,2016,1500,0
4,NST5,1,22,1,1,2017,1500,0


<br></br>


### Processing driver demographic data


Steps:

1. Remove preselected drivers
2. Reformat code to match data
3. Coverting driving licence from year obtained to of years obtained
4. Normalize km/y


In [25]:
driver_demographic_data = process_driver_demographic_data(
    driver_demographic_data, drivers_to_exclude
)

In [26]:
driver_demographic_data.head()

,code,sex,age,mothertongue,education,driving_license,km_year,accidents,NDRT
0,NST01,1,19,1,1,1,200,1,False
1,ST02,1,19,1,1,1,5000,0,True
2,NST03,1,19,1,1,1,1000,0,False
3,ST04,1,21,3,2,2,1500,0,True
4,NST05,1,22,1,1,1,1500,0,False


<br></br>


# Constructing Sequence of Observations


---


The idea is to train 2 HMM trained to observations assosiated with a 'slow' takeover, and a 'fast' takeover.


In [27]:
slow_observations, fast_observations, observations = construct_observations(
    driving_data_dictionary,
    phsyiological_data_dictionary,
    driving_timestamps,
    physio_timestamps,
    driver_demographic_data,
)

In [28]:
print(observations)

Index(['SteeringWheelAngle', 'VehicleSpeed', 'ECG_Clean', 'ECG_Rate',
       'ECG_Quality', 'ECG_R_Peaks', 'ECG_P_Peaks', 'ECG_P_Onsets',
       'ECG_P_Offsets', 'ECG_Q_Peaks', 'ECG_R_Onsets', 'ECG_R_Offsets',
       'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_T_Onsets', 'ECG_T_Offsets',
       'ECG_Phase_Atrial', 'ECG_Phase_Completion_Atrial',
       'ECG_Phase_Ventricular', 'ECG_Phase_Completion_Ventricular',
       'RSP_Clean', 'RSP_Amplitude', 'RSP_Rate', 'RSP_RVT', 'RSP_Phase',
       'RSP_Phase_Completion', 'RSP_Symmetry_PeakTrough',
       'RSP_Symmetry_RiseDecay', 'RSP_Peaks', 'RSP_Troughs', 'EDA_Clean',
       'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks', 'SCR_Height',
       'SCR_Amplitude', 'SCR_RiseTime', 'SCR_Recovery', 'SCR_RecoveryTime',
       'RSA_P2T', 'RSA_Gates', 'code', 'sex', 'age', 'mothertongue',
       'education', 'driving_license', 'km_year', 'accidents', 'NDRT'],
      dtype='object')


<br>
<br>


# Training the HMMs


---


**Train/Validate/Test Split**

In [29]:
slow_observations_train, slow_observations_test = train_test_split(
    slow_observations, test_size=0.1
)

fast_observations_train, fast_observations_test = train_test_split(
    fast_observations, test_size=0.1
)

<br>
<br>


# Hyperparameters


---


In [30]:
# initializing the hyperparameters
n_components = np.arange(1, 11)
covariance_type = ["full", "tied", "diag", "spherical"]
tol = np.arange(0.001, 0.011, 0.001)
init_params = ["kmeans", "k-means++", "random", "random_from_data"]
random_state = np.arange(0, 11)
max_iter = np.linspace(100, 10000, 100).astype(int)

hyperparametes = {
    "n_components": n_components,
    "covariance_type": covariance_type,
    "tol": tol,
    "init_params": init_params,
    "random_state": random_state,
    "max_iter": max_iter,
}

# initialize the model
slow_model = GaussianMixture(reg_covar=1e-4)
fast_model = GaussianMixture(reg_covar=1e-4)

# initialize the random search
slow_random_search = RandomizedSearchCV(
    slow_model, hyperparametes, n_iter=1000, cv=5, n_jobs=-1, error_score='raise'
)
fast_random_search = RandomizedSearchCV(
    fast_model, hyperparametes, n_iter=1000, cv=5,  n_jobs=-1, error_score='raise'
)

# fit the model
s = np.vstack(slow_observations_train)
slow_random_search.fit(s)
f = np.vstack(fast_observations_train)
fast_random_search.fit(f)

/opt/miniconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=GaussianMixture(reg_covar=0.0001), n_iter=1000,
                   n_jobs=-1,
                   param_distributions={'covariance_type': ['full', 'tied',
                                                            'diag',
                                                            'spherical'],
                                        'init_params': ['kmeans', 'k-means++',
                                                        'random',
                                                        'random_from_data'],
                                        'max_iter': array([  100,   200,   300,   400,   500,   600,   700,   800,   900,
        1000,  1100,  1200,  1300,  1400,  1500,  1600,  1700,  1800,
        19...
        6400,  6500,  6600,  6700,  6800,  6900,  7000,  7100,  7200,
        7300,  7400,  7500,  7600,  7700,  7800,  7900,  8000,  8100,
        8200,  8300,  8400,  8500,  8600,  8700,  8800,  8900,  9000,
        9100,  9200,  9300,  9400,  9500,  9600,  9700,  9800,  9900,
       10000]),
                                        'n_components': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'random_state': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'tol': array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009,
       0.01 ])})

In [31]:
slow_model = slow_random_search.best_estimator_
fast_model = fast_random_search.best_estimator_

slow_model.fit(s)
fast_model.fit(f)

GaussianMixture(init_params='random', max_iter=7200, n_components=5,
                random_state=5, reg_covar=0.0001, tol=0.002)

In [32]:
# test the HMM
accuracy = 0

for obs in slow_observations_test:
    if slow_model.score(obs) > fast_model.score(obs):
        accuracy += 1

for obs in fast_observations_test:
    if fast_model.score(obs) > slow_model.score(obs):
        accuracy += 1

accuracy = accuracy / (len(slow_observations_test) + len(fast_observations_test))
print("Accuracy: ", accuracy)

Accuracy:  0.5161290322580645


In [20]:
"""
# initialize the grid search cv
slow_grid = GridSearchCV(slow_model, hyperparametes, cv=5, n_jobs=-1, verbose=1)
fast_grid = GridSearchCV(fast_model, hyperparametes, cv=5, n_jobs=-1, verbose=1)

# fit the model
s = np.vstack(slow_observations_train)
slow_grid.fit(s)
f = np.vstack(fast_observations_train)
fast_grid.fit(f)

# get the best estimators
slow_hmm = slow_grid.best_estimator_
fast_hmm = fast_grid.best_estimator_

slow_hmm.fit(s)
fast_hmm.fit(f)

# test the HMM
accuracy = 0

for obs in slow_observations_test:
    if slow_hmm.score(obs) > fast_hmm.score(obs):
        accuracy += 1

for obs in fast_observations_test:
    if fast_hmm.score(obs) > slow_hmm.score(obs):
        accuracy += 1

accuracy = accuracy / (len(slow_observations_test) + len(fast_observations_test))
print("Accuracy: ", accuracy)
"""

'\n# initialize the grid search cv\nslow_grid = GridSearchCV(slow_model, hyperparametes, cv=5, n_jobs=-1, verbose=1)\nfast_grid = GridSearchCV(fast_model, hyperparametes, cv=5, n_jobs=-1, verbose=1)\n\n# fit the model\ns = np.vstack(slow_observations_train)\nslow_grid.fit(s)\nf = np.vstack(fast_observations_train)\nfast_grid.fit(f)\n\n# get the best estimators\nslow_hmm = slow_grid.best_estimator_\nfast_hmm = fast_grid.best_estimator_\n\nslow_hmm.fit(s)\nfast_hmm.fit(f)\n\n# test the HMM\naccuracy = 0\n\nfor obs in slow_observations_test:\n    if slow_hmm.score(obs) > fast_hmm.score(obs):\n        accuracy += 1\n\nfor obs in fast_observations_test:\n    if fast_hmm.score(obs) > slow_hmm.score(obs):\n        accuracy += 1\n\naccuracy = accuracy / (len(slow_observations_test) + len(fast_observations_test))\nprint("Accuracy: ", accuracy)\n'

Initial Accuracy Including Every Feature:
51.61%
